<a href="https://colab.research.google.com/github/yeoncw1226/2024_2_dataScienceProject_team5/blob/main/DSP_ClusteringPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 랜덤포인트 지정, 서울 소재 대학교별 유학생 수, 서울 구별 공시지가 평균

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 서울_자치구_경계_2017.geojson to 서울_자치구_경계_2017.geojson


In [ ]:
from google.colab import files
import geopandas as gpd
import random
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import triangulate

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 가져오기
file_name = list(uploaded.keys())[0]

# GeoJSON 파일 읽기
gdf = gpd.read_file(file_name)

# 삼각형 기반 랜덤 포인트 생성 함수
def random_point_in_triangle(triangle):
    """삼각형 내부에 랜덤 점 생성"""
    p1, p2, p3 = triangle.exterior.coords[:3]
    r1, r2 = random.random(), random.random()
    if r1 + r2 > 1:
        r1, r2 = 1 - r1, 1 - r2
    x = p1[0] + r1 * (p2[0] - p1[0]) + r2 * (p3[0] - p1[0])
    y = p1[1] + r1 * (p2[1] - p1[1]) + r2 * (p3[1] - p1[1])
    return Point(x, y)

# 다각형 내부에서 랜덤 점 생성 함수
def random_point_in_polygon(polygon):
    """다각형 내부에 랜덤 점 생성"""
    triangles = triangulate(polygon)  # 다각형을 삼각형으로 분할
    areas = [tri.area for tri in triangles]  # 각 삼각형 면적 계산
    area_sum = sum(areas)
    probabilities = [area / area_sum for area in areas]  # 면적 비율로 가중치 계산

    # 삼각형을 선택하여 내부에서 랜덤 점 생성
    chosen_triangle = random.choices(triangles, weights=probabilities, k=1)[0]
    return random_point_in_triangle(chosen_triangle)

# 구별로 랜덤 포인트 n개 생성
n = 4  # 구별로 생성할 랜덤 포인트 개수
random_points_dict = {}

for index, row in gdf.iterrows():
    geometry = row.geometry
    sig_kor_nm = row.get('SIG_KOR_NM', f"Row {index}")  # 지역 이름 확인
    points = []
    if isinstance(geometry, MultiPolygon):
        for _ in range(n):
            poly = random.choice(geometry.geoms)  # MultiPolygon 중 하나 선택
            point = random_point_in_polygon(poly)
            points.append((point.x, point.y))
    elif isinstance(geometry, Polygon):
        for _ in range(n):
            point = random_point_in_polygon(geometry)
            points.append((point.x, point.y))
    random_points_dict[sig_kor_nm] = points

# 결과 출력
print('{')
for key, value in random_points_dict.items():
    print(f"{key}: {value}")
    print(',')
print('}')

Saving 서울_자치구_경계_2017.geojson to 서울_자치구_경계_2017 (5).geojson
{
종로구: [(126.99473848794244, 37.596971726876205), (126.99552826040565, 37.59047403378035), (126.97167302749465, 37.590004653994214), (127.02197446575849, 37.577530263647745)]
,
중구: [(127.0036488165364, 37.56610840576994), (127.00335989140757, 37.54549861427794), (126.9704086427891, 37.55690639019035), (126.99318012695707, 37.55736315219676)]
,
용산구: [(126.98691000613029, 37.537429312201056), (127.0012825038303, 37.52432571140094), (127.00833806569739, 37.52660213303166), (126.9771076169991, 37.50716135833724)]
,
성동구: [(127.06806505620692, 37.55073351302907), (127.04598694478932, 37.53972741096193), (127.03913154548827, 37.55955593384726), (127.07004264326918, 37.55925287997523)]
,
광진구: [(127.07289714783772, 37.5243240226372), (127.07803683021342, 37.54371620722823), (127.08555826488384, 37.55505333153704), (127.08043458474582, 37.54944254711437)]
,
동대문구: [(127.02772439752911, 37.579478444897426), (127.0527244576583, 37.58611081

In [ ]:
from google.colab import files
import pandas as pd

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 가져오기
file_name = list(uploaded.keys())[0]

# 데이터 불러오기
df_cleaned = pd.read_excel(file_name, header=2)

# 열 이름 지정 및 정리
df_cleaned.columns = ['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '계',
                      '학위과정(대학)', '소계(A)', '인문사회계열', '자연과학계열',
                      '공학계열', '예체능계열', '의학계열', '비학위과정', '기타1',
                      '기타2', '기타3', '기타4', '기타5', 'TOPIK충족', '영어트랙',
                      '영어권유학생', '언어능력충족비율', '기숙사수용여부', '계(F)',
                      '수용', '미수용', '계(G)', '기타6']

# "지역"이 "서울"인 데이터 필터링
df_seoul = df_cleaned[df_cleaned['지역'] == '서울']
df_seoul = df_seoul[df_seoul['학교종류'] == '대학교']
# 대학교 이름과 "계" 값을 딕셔너리로 변환
result_dict = df_seoul.set_index('학교')['계'].to_dict()

# 결과 확인
result_dict

Saving 대학교별유학생수.xlsx to 대학교별유학생수 (2).xlsx


{'가톨릭대학교 _제3캠퍼스': 6,
 '감리교신학대학교': 1,
 '강서대학교': 516,
 '건국대학교': 2185,
 '경기대학교 _제2캠퍼스': 564,
 '경희대학교': 4302,
 '고려대학교': 3358,
 '광운대학교': 989,
 '국민대학교': 1951,
 '덕성여자대학교': 483,
 '동국대학교': 1833,
 '동덕여자대학교': 225,
 '명지대학교 _제2캠퍼스': 1934,
 '삼육대학교': 271,
 '상명대학교': 418,
 '서강대학교': 2066,
 '서경대학교': 753,
 '서울과학기술대학교': 669,
 '서울기독대학교': 3,
 '서울대학교': 1452,
 '서울시립대학교': 737,
 '서울여자대학교': 526,
 '성공회대학교': 24,
 '성균관대학교': 3927,
 '성신여자대학교': 510,
 '세종대학교': 1704,
 '숙명여자대학교': 262,
 '숭실대학교': 1924,
 '연세대학교': 4142,
 '이화여자대학교': 1417,
 '장로회신학대학교': 3,
 '중앙대학교': 3097,
 '총신대학교': 112,
 '추계예술대학교': 29,
 '한국성서대학교': 136,
 '한국외국어대학교': 2563,
 '한국체육대학교': 6,
 '한성대학교': 380,
 '한양대학교': 3451,
 '홍익대학교': 1562}

In [ ]:
from google.colab import files
import pandas as pd

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 가져오기
file_name = list(uploaded.keys())[0]

# 필요한 열만 지정하여 데이터 불러오기
columns_to_use = ['시군구명', '공시지가(원/㎡)']
data = pd.read_csv(file_name, encoding='euc-kr', usecols=columns_to_use)

# 공시지가 열을 숫자로 변환
data['공시지가(원/㎡)'] = pd.to_numeric(data['공시지가(원/㎡)'], errors='coerce')

# 시군구명별 공시지가 평균 계산
avg_land_price_by_district = data.groupby('시군구명')['공시지가(원/㎡)'].mean()

# 결과 출력
print(avg_land_price_by_district)

# 딕셔너리로 변환 (원한다면)
avg_land_price_dict = avg_land_price_by_district.to_dict()

Saving 공시지가_2024년.csv to 공시지가_2024년.csv
시군구명
강남구     1.192294e+07
강동구     4.136539e+06
강북구     2.642841e+06
강서구     2.847634e+06
관악구     3.986627e+06
광진구     4.371127e+06
구로구     2.900773e+06
금천구     2.930902e+06
노원구     2.559028e+06
도봉구     2.390374e+06
동대문구    3.783838e+06
동작구     4.280654e+06
마포구     6.174009e+06
서대문구    3.771586e+06
서초구     7.218511e+06
성동구     4.884766e+06
성북구     3.190132e+06
송파구     5.971945e+06
양천구     3.399776e+06
영등포구    4.503093e+06
용산구     6.812532e+06
은평구     3.166812e+06
종로구     5.937878e+06
중구      9.949269e+06
중랑구     2.983133e+06
Name: 공시지가(원/㎡), dtype: float64


In [ ]:
landCost = {
 '강남구': 11922940,
 '강동구': 4136539,
 '강북구': 2642841,
 '강서구': 2847634,
 '관악구': 3986627,
 '광진구': 4371127,
 '구로구': 2900773,
 '금천구': 2930902,
 '노원구': 2559028,
 '도봉구': 2390374,
 '동대문구': 3783838,
 '동작구': 4280654,
 '마포구': 6174009,
 '서대문구': 3771586,
 '서초구': 7218511,
 '성동구': 4884766,
 '성북구': 3190132,
 '송파구': 5971945,
 '양천구': 3399776,
 '영등포구': 4503093,
 '용산구': 6812532,
 '은평구': 3166812,
 '종로구': 5937878,
 '중구': 9949269,
 '중랑구': 2983133
}
#공시지가 평균

In [ ]:
import pandas as pd
import requests

# Kakao API 키 설정
KAKAO_API_KEY = "87b40cb5d8022b260431de3f207f259a"

# 주소로 좌표를 반환하는 함수
def get_coordinates(address):
  url = "https://dapi.kakao.com/v2/local/search/address.json"
  headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
  params = {"query": address}

  response = requests.get(url, headers=headers, params=params)

  if response.status_code == 200:
    result = response.json()
    if result['documents']:
      # 첫 번째 결과의 위도와 경도 반환
      latitude = result['documents'][0]['y']
      longitude = result['documents'][0]['x']
      return latitude, longitude
  # 실패 시 None 반환
  return None, None

# 데이터 불러오기
file_path = '/content/drive/MyDrive/police.csv'
df = pd.read_csv(file_path, encoding='cp949')  # 파일 확장자에 맞게 read_csv 사용

# 열 이름 확인
print("열 이름:", df.columns)

# 올바른 주소 열 이름 지정 (여기서 '주소', '지역경찰관서', '신주소' 중 하나를 사용)
address_column = '주소'  # 열 이름 확인 후 수정 필요

# 위도와 경도 컬럼 추가
df['위도'] = None
df['경도'] = None

# 각 주소에 대해 위도와 경도 검색
for index, row in df.iterrows():
  address = row[address_column]  # 지정한 열 이름 사용
  if pd.notnull(address):  # 주소가 유효한 경우에만 API 호출
    latitude, longitude = get_coordinates(address)
    df.at[index, '위도'] = latitude
    df.at[index, '경도'] = longitude

# 수정된 데이터를 동일 파일에 저장
df.to_csv(file_path, index=False, encoding='utf-8-sig')  # UTF-8로 저장

print(f"파일이 수정되었습니다: {file_path}")


열 이름: Index(['경찰서', '주소'], dtype='object')
파일이 수정되었습니다: /content/drive/MyDrive/police.csv
